In [1]:
import pandas as pd
import zipfile
from IPython.display import display, HTML


navegação

In [2]:
from DT_atualiza_settings import *


url = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm'
driver.get(url)

popup = '//*[@id="onetrust-close-btn-container"]/button' ## '/html/body/div[2]/div[3]/div/div[2]/button'
driver.find_element(By.XPATH, popup).click()


from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "bvmf_iframe")))

botao_expandir = '/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[1]/div/div/a/i'
driver.find_element(By.XPATH, botao_expandir).click()
# driver.execute_script("arguments[0].click();", driver.find_element('xpath', botao_expandir))

caminho='/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[2]/div/app-companies-home-filter-classification/form/div[2]/div[3]/div[2]/p/a'
planilha = driver.find_element(By.XPATH, caminho)
driver.execute_script("arguments[0].click();", planilha)

sleep(5)



extrair o ZIP

In [3]:
arquivo_zip = zipfile.ZipFile( data_path + r"ClassifSetorial.zip")

for planilha in arquivo_zip.namelist():
    setores = pd.read_excel(arquivo_zip.open(planilha), skiprows=6)

arquivo_zip.close()

setores.rename(columns={'Unnamed: 4': 'SM'}, inplace=True)
setores.head(6)


,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,SM
0,NaN,NaN,NaN,CÓDIGO,SEGMENTO
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN
2,NaN,NaN,3R PETROLEUM,RRRP,NM
3,NaN,NaN,COSAN,CSAN,NM
4,NaN,NaN,PET MANGUINH,RPMG,NaN
5,NaN,NaN,PETROBRAS,PETR,N2


limpeza

In [4]:
# completar as linhas das duas primerios colunas trocando NAN pelo "anterior"
setores['SETOR ECONÔMICO'] = setores['SETOR ECONÔMICO'].ffill()
setores['SUBSETOR'] = setores['SUBSETOR'].ffill()
# remover linhas NaN NaN NaN xxx yyy
setores = setores.dropna(subset=['SETOR ECONÔMICO','SUBSETOR','SEGMENTO'], how='all')
# remover linhas xxx yyy NaN NaN NaN
setores = setores.dropna(subset=['SEGMENTO','LISTAGEM','SM'], how='all')
setores.head()

,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,SM
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN
2,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",3R PETROLEUM,RRRP,NM
3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",COSAN,CSAN,NM
4,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",PET MANGUINH,RPMG,NaN
5,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",PETROBRAS,PETR,N2


In [5]:
# remove linhas com conteudo
# 'SETOR ECONÔMICO' 'SUBSETOR' 'SEGMENTO' 'LISTAGEM' 'NaN'

condicao1 = (setores['SETOR ECONÔMICO'] == 'SETOR ECONÔMICO') & \
           (setores['SUBSETOR'] == 'SUBSETOR') & \
           (setores['SEGMENTO'] == 'SEGMENTO') & \
           (setores['LISTAGEM'] == 'LISTAGEM') & \
           (setores['SM'].isna())
# Removendo as linhas que correspondem à condição
setores = setores[~condicao1]
# remove linhas com conteudo
# 'SETOR ECONÔMICO'	'SUBSETOR'	'NaN'	'CÓDIGO'	'SEGMENTO'
condicao2 = (setores['SETOR ECONÔMICO'] == 'SETOR ECONÔMICO') & \
           (setores['SUBSETOR'] == 'SUBSETOR') & \
           (setores['SEGMENTO'].isna()) & \
           (setores['LISTAGEM'] == 'CÓDIGO') & \
           (setores['SM'] == 'SEGMENTO')

# Removendo as linhas que correspondem à condição
setores = setores[~condicao2]

In [6]:
setores_html =setores.copy()
# Função para aplicar o estilo condicional a toda a coluna 3
def formatar_negrito(cell_value, row):
    if pd.isna(row['LISTAGEM']) and pd.isna(row['SM']):
        return f'<b><i>{cell_value}</i></b>'
    return cell_value

# Aplicar a formatação na coluna 3
setores_html['SETOR ECONÔMICO'] = setores.apply(lambda row: formatar_negrito(row['SETOR ECONÔMICO'], row), axis=1)
setores_html['SUBSETOR'] = setores.apply(lambda row: formatar_negrito(row['SUBSETOR'], row), axis=1)
setores_html['SEGMENTO'] = setores.apply(lambda row: formatar_negrito(row['SEGMENTO'], row), axis=1)

# Exibir o DataFrame formatado em HTML
display(HTML(setores_html.to_html(escape=False, index=False)))

SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,SM
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",3R PETROLEUM,RRRP,NM
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",COSAN,CSAN,NM
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",PET MANGUINH,RPMG,NaN
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",PETROBRAS,PETR,N2
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",PETRORECSA,RECV,NM
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",PETRORIO,PRIO,NM
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",RAIZEN,RAIZ,N2
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",ULTRAPAR,UGPA,NM
"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",VIBRA,VBBR,NM


In [7]:
# SETOR ECONÔMICO	SUBSETOR	SEGMENTO	LISTAGEM	SM
# Símbolo a ser procurado
target_symbol = 'GOAU'

# Encontrando o índice do símbolo
target_index = setores[setores['LISTAGEM'] == target_symbol].index[0]

# Buscando símbolos acima do alvo até o primeiro NaN
symbols_above = []
for i in range(target_index - 1, -1, -1):
    if pd.isna(setores.loc[i, 'LISTAGEM']):
        break
    symbols_above.append(setores.loc[i, 'LISTAGEM'])

# Buscando símbolos abaixo do alvo até o primeiro NaN
symbols_below = []
for i in range(target_index + 1, len(setores)):
    if pd.isna(setores.loc[i, 'LISTAGEM']):
        break
    symbols_below.append(setores.loc[i, 'LISTAGEM'])

# Resultados combinados
symbols_nearby = symbols_above[::-1] + [target_symbol] + symbols_below

# Exibindo os resultados
print("Símbolos próximos a 'GOAU':", symbols_nearby)

Símbolos próximos a 'GOAU': ['FESA', 'GGBR', 'GOAU', 'CSNA', 'USIM']


In [8]:
os.remove(data_path + r"ClassifSetorial.zip")
## remove planilha caso exista
try:
    os.remove(data_path + "setores.csv")
except: pass

driver.quit()

In [9]:
setores.to_csv(data_path+"setores.csv", sep=';' , index = False)